# Load data

In [ ]:
data_key = sys.argv[1]
DB = db_pck.load_db(env_var_db_path)
data_path = load_path(DB.fetch_path(data_key))
dataset = load_dataset(data_path)

# Fisheye correction

In [ ]:
video = dataset.GetVideo()
c_video = cv.CorrectVideo(video, dataset.meta.camera_matrix)

# Pose extraction

In [ ]:
pose_info = GetPose(c_video)

# Unit vector extraction

In [ ]:
points2D = ExtractPoints(pose_info)
unit_vectors = PointsToUnitVectors(points2D, dataset.meta.camera_frame)

# Conversion to 3D

In [ ]:
points3D = LeastSquares(unit_vectors, dataset.meta.rider_info.GetConstPairs())

# Points to Area

In [ ]:
polyline = Fit3DMesh(points3D, dataset.meta.rider_info.GetMesh())

# Projection to Plane

In [ ]:
plane = create_plane(dataset.wind_direction)
polyshape = ProjectToPlane(polyline)

# Calculate CdA

In [ ]:
area = GetArea(polyshape)
CdA = CalculateCdA(area)
drag = CalculateDrag(dataset.velocity, CdA)
rel_power = CalculateRelative(dataset.power, drag)

# Visualize

In [ ]:
play_video(c_video)
overlay(pose_info)
overlay(polyshape)

draw3D(points3D)
draw3D(polyline)
draw2D(polyshape)

drawLabel(area, CdA, drag, rel_power, dataset.velocity, dataset.power)
drawGraphs(...)